In [1]:
from lib import *
import numpy as np

import os
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from plotly.subplots import make_subplots
from itertools import product

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [2]:
data1,key_list = load_larsoft_root("RAW","data/full_1rms/opdetraw_1rms_hist.root", ["opdigi","opdigiana"],np.loadtxt("template/fbk_deco.txt"),label="RAW")
# data1,key_list = load_larsoft_root("RAW","data/test/opdetraw_ideal_hist.root", ["opdigi","opdigiana"],np.loadtxt("template/fbk_deco.txt"),label="RAW")
data2,key_list = load_larsoft_root("DEC","data/full_1rms/deconv_gauss_1rms_hist.root", ["opdigi","opdecoana"  ],np.loadtxt("template/fbk_deco.txt"),label="GAUSS")
data3,key_list = load_larsoft_root("DEC","data/full_1rms/deconv_wiener_1rms_hist.root",["opdigi","opdecoana"  ],np.loadtxt("template/fbk_deco.txt"),label="WIENER")

color_map={data1["LABEL"]: "#3366CC", data2["LABEL"]: "#66AA00", data3["LABEL"]:"#FF9900"}


In [3]:
raw_reco1 = ophit_data("data/test/ophitspe_area410.root","RAW")
reco1 = order_ophit_data(raw_reco1,data1["RECO"]["CH"])
raw_reco2 = ophit_data("data/test/ophitspe_gauss_slidingwindow_hist.root","GAUSS")
reco2 = order_ophit_data(raw_reco2,data2["RECO"]["CH"])
raw_reco3 = ophit_data("data/test/ophitspe_wiener_slidingwindow_hist.root","WIENER")
reco3 = order_ophit_data(raw_reco3,data3["RECO"]["CH"])

ophit_1 = combine_ophit_with_data(data1,reco1)
ophit_2 = combine_ophit_with_data(data2,reco2)
ophit_3 = combine_ophit_with_data(data3,reco3)

['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']


*** VISUALIZE DATA ***

In [4]:
# Choose between a random channel number or select yourself
# ch = 218
ch = np.random.choice(data1["RECO"]["CH"])
num = np.where(np.asarray(data2["RECO"]["CH"]) == ch)[0][0]
# peak_times = reco2[ch]["PeakTime"]
# peak_amps = reco2[ch]["Amplitude"]/500
# ophitfinder_pe = np.sum(reco2[ch]["PE"])

fig = make_subplots(rows=1, cols=3,subplot_titles=('', ''))

fig.add_trace(go.Scatter(line=dict(color=color_map[data1["LABEL"]]),name="%s #PE: %.2f"%(data1["LABEL"],data1["RECO"]["PE"][num]),x=data1["RECO"]["WVF_X"][num],y=data1["RECO"]["WVF"][num]-data1["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data2["LABEL"]]),name="%s #PE: %.2f"%(data2["LABEL"],data2["RECO"]["PE"][num]),x=data2["RECO"]["WVF_X"][num],y=data2["RECO"]["WVF"][num]),col=2,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data3["LABEL"]]),name="%s #PE: %.2f"%(data3["LABEL"],data3["RECO"]["PE"][num]),x=data3["RECO"]["WVF_X"][num],y=data3["RECO"]["WVF"][num]),col=3,row=1)

fig.add_vline(x=data1["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=1)
fig.add_vline(x=data2["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=2)
fig.add_vline(x=data3["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=3)

try:
    if len(data1["TRUE"]["PETIMES"][num]) > 1000:
        fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6),col=1,row=1)
    else:
        fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6,y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=1,row=1)
        fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6,y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=2,row=1)
        fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6,y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=3,row=1)
except:
    print("WVF WITHOUT TRUE PHOTONS")

fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(np.sum(reco1[ch]["PE"])),line=dict(color="#DC3912"),mode="markers",x=reco1[ch]["PeakTime"],y=reco1[ch]["Amplitude"]),col=1,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(np.sum(reco2[ch]["PE"])),line=dict(color="#DC3912"),mode="markers",x=reco2[ch]["PeakTime"],y=reco2[ch]["Amplitude"]/500),col=2,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(np.sum(reco3[ch]["PE"])),line=dict(color="#DC3912"),mode="markers",x=reco3[ch]["PeakTime"],y=reco3[ch]["Amplitude"]/500),col=3,row=1)


fig.update_layout(title="Deco. wvf comparison for ch %i"%ch,xaxis_title="Time in [&mu;s]",yaxis_title="Amp. in [ADC]",xaxis2_title="Time in [&mu;s]",yaxis2_title="Amp. in [a.u.]",xaxis3_title="Time in [&mu;s]",yaxis3_title="Amp. in [a.u.]")
fig.update_layout(autosize=True,font=dict(size=14))

fig.show()


*** CONFIGURE FILTER FOR ANALYSIS ***

In [5]:


pe_error_1 = (np.asarray(data1["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
pe_error_2 = (np.asarray(data2["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
pe_error_3 = (np.asarray(data3["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])

ophitpe_error_1 = (np.asarray(ophit_1["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
ophitpe_error_2 = (np.asarray(ophit_2["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
ophitpe_error_3 = (np.asarray(ophit_3["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])

t0_reco_1 = np.asarray(data1["RECO"]["T0"])-data1["PRETRIGGER"]*data1["SAMPLING"]*1e6
t0_reco_2 = np.asarray(data2["RECO"]["T0"])-data1["PRETRIGGER"]*data1["SAMPLING"]*1e6
t0_reco_3 = np.asarray(data3["RECO"]["T0"])-data1["PRETRIGGER"]*data1["SAMPLING"]*1e6

amp           = np.concatenate([data1["RECO"]["AMP"],data2["RECO"]["AMP"],data3["RECO"]["AMP"]])
pe_reco       = np.concatenate([data1["RECO"]["PE"],data2["RECO"]["PE"],data3["RECO"]["PE"]])
pe_true       = np.concatenate([data1["TRUE"]["PE"],data1["TRUE"]["PE"],data1["TRUE"]["PE"]])
pe_ophit      = np.concatenate([ophit_1["PE"],ophit_2["PE"],ophit_3["PE"]])
ophitpe_error = np.concatenate([ophitpe_error_1,ophitpe_error_2,ophitpe_error_3])
pe_error      = 100*np.concatenate([pe_error_1,pe_error_2,pe_error_3])
t0_reco       = np.concatenate([t0_reco_1,t0_reco_2,t0_reco_3])
filter_label  = np.concatenate([[data1["LABEL"]]*len(pe_error_1),[data2["LABEL"]]*len(pe_error_2),[data3["LABEL"]]*len(pe_error_3)])

df = pd.DataFrame({"FILTER":filter_label, "'%'ERROR PE":pe_error, "'%'ERROR OPHIT PE":ophitpe_error, "TRUE PE":pe_true, "RECO PE":pe_reco, "OPHIT PE":pe_ophit, "AMP [a.u.]":amp, "RECO T0":t0_reco})

In [6]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 1000)], y='TRUE PE', x="AMP [a.u.]",
color='FILTER',color_discrete_map=color_map, marginal_x='histogram',trendline="ols")
fig.update_layout(autosize=False,width=1000,height=800,font=dict(size=16))
fig.show()

results = px.get_trendline_results(fig)
# print(results)
results.query("FILTER == 'GAUSS'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                 1.999e+04
Date:                Tue, 21 Mar 2023   Prob (F-statistic):               0.00
Time:                        16:36:01   Log-Likelihood:                -2356.0
No. Observations:                 461   AIC:                             4716.
Df Residuals:                     459   BIC:                             4724.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.8654      2.387     12.090      0.000      24.174      33.557
x1            10.5015      0.074    141.383      0.000      10.356      10.648
==============================================================================
Omnibus:                       30.048   Durbin-Watson:                   1.223
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              108.250
Skew:                           0.059   Prob(JB):                     3.12e-24
Kurtosis:                       5.371   Cond. No.                         41.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 2000)], y="TRUE PE", x="'%'ERROR OPHIT PE",
color='FILTER',color_discrete_map=color_map,marginal_x='box',template="plotly")
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.update_layout(autosize=False,width=1500,height=800,font=dict(size=16))
fig.update_traces(opacity=0.75)
fig.show()

In [8]:
fig = px.histogram(data_frame=df[(df['TRUE PE'] > 0)*(df['TRUE PE'] < 1000)], x="RECO T0",
color='FILTER',color_discrete_map=color_map,barmode="group",marginal='box',template="plotly",nbins=3000,range_x=[-0.16,0.32])
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.update_layout(autosize=False,width=1500,height=800,font=dict(size=16))

fig.show()